<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [ ]:
import numpy as np

**Helper Functions**

In [ ]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C^(-1)Y
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1) + np.dot(Y, V.T.conj())
    # calculate and return C^(-1)Y
    return np.dot(np.linalg.inv(C), y)

**RQI Algorithm**

In [ ]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # keep track of iterations
    i = 0
    # conditional stop variable
    stop = False
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    while(not stop):
        # increase iteration count
        i += 1
        # check that max iterations hasn't been passed
        # if it has been passed, return False to indicate failure
        # also return the max number of iterations
        if i > it:
            return False, False, it
        # compute the dot product of ((mu)I-M)^(-1) and y
        invMat = np.linalg.inv(A(mu, M))
        x = np.dot(invMat, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, and i
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i


**RQI Algorithm with Additive Preconditioning**

In [ ]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # keep track of iterations
    i = 0
    # conditional stop variable
    stop = False
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    while(not stop):
        # increase iteration count
        i += 1
        # check that max iterations hasn't been passed
        # if it has been passed, return False to indicate failure
        # also return the max number of iterations
        if i > it:
            return False, False, it
        # compute f()
        x = f(mu, M, y, v)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, and i
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i

**Test and Compare Algorithms**

In [ ]:
# Setup

# create a random matrix M
M = np.random.rand(4, 4)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [ ]:
# Test RQI with 1e-4
RQI(M, mu, 1e-4, 100)

(2.4263605049974886,
 array([0.61032257, 0.30146674, 0.42207949, 0.5987262 ]),
 3)

In [ ]:
# Test RQIAPP with 1e-4
RQIAPP(M, mu, 1e-4, 100)

(2.4263605165584172,
 array([0.61032258, 0.30146676, 0.4220795 , 0.59872618]),
 3)

In [ ]:
# Test RQI with 1e-6
RQI(M, mu, 1e-6, 100)

(2.426360504951676, array([0.61032257, 0.30146674, 0.42207949, 0.5987262 ]), 4)

In [ ]:
# Test RQIAPP with 1e-6
RQIAPP(M, mu, 1e-6, 100)

(2.426360513179198, array([0.61032257, 0.30146676, 0.42207949, 0.59872619]), 3)

In [ ]:
# Test RQI with 1e-8
RQI(M, mu, 1e-8, 100)

(False, False, 100)

In [ ]:
# Test RQIAPP with 1e-8
RQIAPP(M, mu, 1e-8, 100)

(2.426360504951676, array([0.61032257, 0.30146674, 0.42207949, 0.5987262 ]), 4)

In [ ]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

2.426360504951673 [-0.61032257 -0.30146674 -0.42207949 -0.5987262 ]
